In [1]:
import pandas as pd
import numpy as np
import json
import os 
from spacy.tokens import DocBin
from tqdm import tqdm
import random

In [1]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda112,transformers,lookups]'
!python -m spacy download en_core_web_trf

  Using cached spacy-3.4.0-cp38-cp38-macosx_10_9_x86_64.whl (6.5 MB)
  Using cached spacy-3.3.1-cp38-cp38-macosx_10_9_x86_64.whl (6.5 MB)
  Using cached thinc-8.0.17-cp38-cp38-macosx_10_9_x86_64.whl (640 kB)
  Using cached spacy-3.3.0-cp38-cp38-macosx_10_9_x86_64.whl (6.4 MB)
  Using cached spacy-3.2.4-cp38-cp38-macosx_10_9_x86_64.whl (6.2 MB)
  Using cached spacy-3.2.3-cp38-cp38-macosx_10_9_x86_64.whl (6.2 MB)
  Using cached spacy-3.2.2-cp38-cp38-macosx_10_9_x86_64.whl (6.2 MB)
  Using cached spacy-3.2.1-cp38-cp38-macosx_10_9_x86_64.whl (6.2 MB)
  Using cached spacy-3.2.0-cp38-cp38-macosx_10_9_x86_64.whl (6.2 MB)
  Using cached spacy-3.1.6-cp38-cp38-macosx_10_9_x86_64.whl (6.1 MB)
  Using cached spacy-3.1.5-cp38-cp38-macosx_10_9_x86_64.whl (6.1 MB)
  Using cached spacy-3.1.4-cp38-cp38-macosx_10_9_x86_64.whl (6.1 MB)
  Using cached spacy-3.1.3-cp38-cp38-macosx_10_9_x86_64.whl (6.0 MB)
  Using cached spacy_transformers-1.0.6-py2.py3-none-any.whl (42 kB)
  Using cached spacy-3.1.2-cp38-c

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.2.0/local_installers/cuda-repo-debian10-11-2-local_11.2.0-460.27.04-1_amd64.deb
!sudo dpkg -i cuda-repo-debian10-11-2-local_11.2.0-460.27.04-1_amd64.deb
!sudo apt-key add /var/cuda-repo-debian10-11-2-local/7fa2af80.pub
!sudo add-apt-repository contrib
!sudo apt-get update
!sudo apt-get -y install cuda-11.2

In [3]:
import spacy

In [ ]:
! nvcc --version

In [7]:
with open('annotations.json') as f:
  data = json.load(f)

with open('annotations_val.json') as f:
  data_val = json.load(f)

train_data = data['annotations']
val_data = data_val['annotations']

In [8]:
def create_training(train_data):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(train_data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label,
                                 alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [ ]:
train_data = create_training(train_data)
train_data.to_disk("train.spacy")
val_data = create_training(val_data)
val_data.to_disk("val.spacy")

In [ ]:
!nvidia-smi

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy debug data config.cfg

In [13]:
!python -m spacy train config.cfg --output output --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-07-10 19:58:12,422] [INFO] Set up nlp object from config
[2022-07-10 19:58:12,431] [INFO] Pipeline: ['transformer', 'ner']
[2022-07-10 19:58:12,435] [INFO] Created vocabulary
[2022-07-10 19:58:12,436] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be

In [7]:
import numpy as np
import pandas as pd

pd = pd.read_csv("facebook_train.csv")

In [34]:
texts = pd.loc[2, 'Description']
texts

" Driven 180000 miles Automatic transmission Exterior color: Silver · Interior color: Grey Fuel type: Gasoline 19.0 MPG city · 26.0 MPG highway · 22.0 MPG combined 2 owners This vehicle is paid off Seller's Description N3ED ABS SENSOR WHICH I HAVE 180K heated leather seats power everything run and drive good   Marlborough MA Location is approximate Marlborough MA Location is approximate Seller Information Seller details James Louis Joined Facebook in 2020"

# New Section

In [35]:
texts = ['Driven 180000 miles Automatic transmission Exterior color: Silver · Interior color: Grey Fuel type: Gasoline 19.0 MPG city · 26.0 MPG highway · 22.0 MPG combined 2 owners This vehicle is paid off Seller\'s Description N3ED ABS SENSOR WHICH I HAVE 180K heated leather seats power everything run and drive good   Marlborough MA Location is approximate Marlborough MA Location is approximate Seller Information Seller details James Louis Joined Facebook in 2020']
nlp = spacy.load('output/model-best')
for doc in nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])

[('Silver', 'EX_COLOR'), ('Grey', 'IN_COLOR'), ('Gasoline', 'FUEL_TYPE'), ('19.0', 'CITY'), ('26.0', 'HIGHWAY'), ('22.0', 'COMBINED'), ('2', 'NUM_ONWER'), ('heated leather seats power everything run and drive good', 'CONDITION_GOOD')]


In [ ]:
 !zip -r output.zip output/
